In [2]:


import tensorflow as tf
import random
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
import numpy as np

from matplotlib import pyplot
import data_generator


In [3]:
# load in the data 

full_dataset = np.load("ml_dataset.npy");

In [4]:
# split up the data into the x and y halves

x_set = full_dataset[0];
y_set = full_dataset[1];

In [30]:
# split the data into training and testing sets


half = int(3*len(x_set)/4);

print(f'half: {half}');
x_train = x_set[0:half];
x_test = x_set[half:];

y_train = y_set[0:half];
y_test = y_set[half:];



half: 74


In [31]:
# print some data diagnostics to ensure we correctly obtained data sets

print(f"x_set shape: {x_set.shape}");
print(f"y_set shape: {y_set.shape}");


training_point_number = x_set.shape[1];
print(f"number of points in each graph set: {training_point_number}");


print("number of x training data sets:");
print(len(x_train[0:]));


print("number of y training data sets:");
print(len(y_train[0:]));

assert(len(x_train) == len(y_train));
assert(len(x_test) == len(y_test));
assert(x_set.shape == y_set.shape);

print(f"\nNumber of training graph-solution sets: {len(x_train)}")


print(f"Number of testing graph-solution sets: {len(x_test)}")



print(f"\nDataset shape: {x_set.shape}")



x_set shape: (99, 500)
y_set shape: (99, 500)
number of points in each graph set: 500
number of x training data sets:
74
number of y training data sets:
74

Number of training graph-solution sets: 74
Number of testing graph-solution sets: 25

Dataset shape: (99, 500)


In [32]:
# reshape the x data into a column of 1 element rows for the LSTM


x_train = x_train.reshape(x_train.shape[0],training_point_number,1);
x_test = x_test.reshape(x_test.shape[0],training_point_number,1);

print(x_train.shape)

(74, 500, 1)


In [33]:
#build the model by stacking layers


model = tf.keras.models.Sequential()

model.add(LSTM(training_point_number));
model.add(Dense(training_point_number,activation='relu'));
model.add(Dropout(0.2));

model.add(Dense(training_point_number,activation='relu'));
model.add(Dropout(0.2));

model.add(Dense(training_point_number,activation='relu'));
model.add(Dropout(0.2));

model.add(Dense(training_point_number,activation='relu'));
model.add(Dropout(0.2));

model.add(Dense(training_point_number,activation='relu'));
model.add(Dropout(0.2));

model.add(Dense(training_point_number,activation='softmax'));


print("model built");


model built


In [34]:
# get the predictions for the second datapoint
predictions = model(x_train)

# print the raw predictions for that datapoint. Note that these are "logits" or "log-odds" scores, which is related to the probability, 
# and also is the inverse of the sigmoid function
print("logits prediction: ");
print(predictions[0]);

# we can use the softmax function to convert these logits to probabilities for each preciction class
print("odds prediction: ");
print(tf.nn.softmax(predictions).numpy()[0]);


print(f'shape: {predictions.shape}');


logits prediction: 
tf.Tensor(
[0.00200001 0.00200001 0.00199998 0.00200008 0.00200003 0.00199999
 0.00200006 0.00200005 0.00200005 0.00199992 0.00200001 0.00199999
 0.00200006 0.00200011 0.00200001 0.00200001 0.00199998 0.002
 0.00200006 0.00200001 0.00199995 0.00199999 0.00200005 0.00200004
 0.00200001 0.00199997 0.00199996 0.00199993 0.00200002 0.00200006
 0.00199998 0.00199998 0.00199982 0.00199998 0.00200006 0.00199991
 0.00199999 0.00200007 0.00200011 0.00199998 0.00199989 0.00199994
 0.00200002 0.00200006 0.00199999 0.002      0.00199998 0.00200001
 0.00200002 0.00200002 0.00199993 0.00200008 0.0019999  0.00199999
 0.00200002 0.00200001 0.002      0.00199997 0.00199996 0.00199992
 0.00200004 0.00200004 0.00200002 0.00200004 0.00200003 0.00200002
 0.00200003 0.00200003 0.0019999  0.00199991 0.00199995 0.00199997
 0.00199998 0.00200007 0.00199995 0.00200011 0.00200004 0.00200003
 0.00200003 0.00199996 0.00199998 0.00200001 0.00200006 0.00199991
 0.00200004 0.00200003 0.00200014 0.

In [35]:
print(y_train[0])

[  106.81238782   128.00362308   149.19485833   170.38609359
   191.57732885   212.76856411   233.95979937   255.15103463
   276.34226988   297.53350514   318.7247404    339.91597566
   361.10721092   382.29844618   403.48968144   424.68091669
   445.87215195   467.06338721   488.25462247   509.44585773
   530.63709299   551.82832824   573.0195635    594.21079876
   615.40203402   636.59326928   657.78450454   678.97573979
   700.16697505   721.35821031   742.54944557   763.74068083
   784.93191609   806.12315135   827.3143866    848.50562186
   869.69685712   890.88809238   912.07932764   933.2705629
   954.46179815   975.65303341   996.84426867  1018.03550393
  1039.22673919  1060.41797445  1081.6092097   1102.80044496
  1123.99168022  1145.18291548  1166.37415074  1187.565386
  1208.75662126  1229.94785651  1251.13909177  1272.33032703
  1293.52156229  1314.71279755  1335.90403281  1357.09526806
  1378.28650332  1399.47773858  1420.66897384  1441.8602091
  1463.05144436  1484.242679

In [36]:
# get the loss function as a MeanSquaredError loss from tf.keras.losses. 
loss_fn = tf.keras.losses.MeanSquaredError();


print("loss:");
print(loss_fn(y_train[0],predictions[0]).numpy());

loss:
38451044.0


In [37]:
opt = tf.keras.optimizers.Adam(learning_rate=1e-5);

model.compile(loss=loss_fn,optimizer=opt,metrics=['accuracy']);


In [38]:
model.fit(x_train,y_train,epochs=10,validation_data=(x_test,y_test), batch_size=10);

Epoch 1/10
8/8 [==============================] - 26s 3s/step - loss: 521727.3438 - accuracy: 0.0000e+00 - val_loss: 3.6115e-06 - val_accuracy: 0.0000e+00
Epoch 2/10
8/8 [==============================] - 24s 3s/step - loss: 521727.3438 - accuracy: 0.0000e+00 - val_loss: 3.6114e-06 - val_accuracy: 0.0000e+00
Epoch 3/10


In [ ]:

print(model.summary());

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 500)               1004000   
_________________________________________________________________
dense (Dense)                (None, 500)               250500    
_________________________________________________________________
dropout (Dropout)            (None, 500)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 500)               250500    
_________________________________________________________________
dropout_1 (Dropout)          (None, 500)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 500)               250500    
_________________________________________________________________
dropout_2 (Dropout)          (None, 500)               0

In [ ]:
predictions = model(x_train);

In [1]:
first_graph = predictions[25];
print(predictions.shape);

x = np.linspace(0,500,500);
print(x.shape)

pyplot.plot(x, first_graph);
pyplot.plot(x,y_set[25])

NameError: name 'predictions' is not defined